# Data Collection & Cleaning, EDA Part 1, Feature Engineering & Analysis

In [271]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, f1_score, fbeta_score, confusion_matrix, classification_report
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [122]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# For auto-reloading updated modules
%load_ext autoreload
%autoreload 2

In [295]:
# Import modules
from data_cleaning import convert
from model_eval import model_scores_os

In [324]:
# Connect to postgres
import psycopg2 as pg

connection_args = {
    'host': 'localhost',
    'dbname': 'covid',
    'port': 5432
}

connection = pg.connect(**connection_args) 

## Data Collection

### SQL Queries

**Set-up:**
Imported .csv files as tables into 'covid' database:
- Patient/case-level data with patient outcome and demographics from CDC (table name: 'cases')
- National-level hospitalizations, ICU admissions, and other metrics from The Atlantic (table name: 'hosp')
- State-level hospital utilization data from CDC (table name: 'inpatient')

#### **Patient/case-level data (CDC)**
- Original size of data set: 4481062 rows, 11 columns
```
SELECT COUNT(*) FROM cases;
SELECT column_name, data_type FROM information_schema.columns WHERE table_name=cases;
```
| column_name | data type |
| ----------- | --------- |
| cdc_report_dt | date |
| pos_spec_dt    | date |
| onset_dt       | date |
| current_status | text |
| sex            | text |
| age_group      | text |
| race_ethnicity | text |
| hosp           | text |
| icu            | text |
| death          | text |
| med_cond       | text |


- The "current status" column indicates whether or not a patient is a laboratory-confirmed vs. probable case. For accuracy purposes, I will filter this dataset for only laboratory-confirmed cases. Since the clinical presentation of COVID overlaps significantly with those of other illnesses, it is difficult to definitively diagnose without laboratory confirmation (source?).
```
CREATE VIEW cases_pos AS SELECT * FROM cases WHERE current_status = 'Laboratory-confirmed case';
```
count: 4322724


- Dropped all rows where target variable ('death') was missing/unknown and created new view
```
CREATE VIEW cases_death AS SELECT * FROM cases_pos WHERE death = 'Yes' OR death ='No';
```
count: 2179638


- I will need a date point for this data set. I want to focus on when the patient was confirmed to have COVID-19 infection, which is indicated by the date of the first positive specimen ('pos_spec_dt'). I will also merge the other data sets on this column since the other data sets contain time-based data. Since it would be difficult to impute other dates and this is my primary date point on which several data points will converge, I plan to drop all rows where this value is null.
- Check the distribution of my target variable in the subset that I plan to drop:
```
SELECT death, COUNT(*) from cases_death WHERE pos_spec_dt IS NULL GROUP BY death;
```
| death | count |
| ----- | ----- |
| No    | 1406147|
| Yes   |   81418|

- The distribution is about the same as in the original.


- Looked at distribution of target variable, 'death':
```
SELECT death, COUNT(*) FROM cases_pos GROUP BY death;
```
| death | count |
| ----- | ----- |
| No    | 2050099
| Yes   |  129539


- Check distinct values and counts for each column:
```
SELECT {column name} FROM cases_deaths GROUP BY {column name};
```


- I want to look for missing/null values. I will do a preliminary analysis to see what kind of information this subset contains.
- Looked at distribution of target variable among cases where any feature was missing/unknown:
```
SELECT death, COUNT(*) FROM cases_deaths WHERE cdc_report_dt IS NULL OR pos_spec_dt IS NULL OR onset_dt IS NULL OR sex = 'Missing' OR sex = 'Unknown' OR sex = 'NA' OR age_group = 'Unknown' OR age_group = 'NA' OR race_ethnicity = 'NA' OR race_ethnicity = 'Unknown' OR hosp = 'Missing' OR hosp = 'Unknown' OR icu = 'Missing' OR icu = 'Unknown' OR med_cond = 'Missing' OR med_cond = 'Unknown' GROUP BY death;
```
| death | count |
| ----- | ----- |
| No    | 1929212 |
| Yes   |  126008 |
 
It looks like the majority of the data contains null values. I cannot drop these rows since I will likely lose valuable data, but I will do some preliminary data/feature analysis to determine which features are likely to be most important.


- Look for any data points that don't make sense. In particular, it wouldn't make sense for a patient who is in the ICU to not be hospitalized. I will drop these rows since the data may not be accurate.
```
DELETE * FROM cases_deaths WHERE hosp = 'No' AND icu = 'Yes';
```

- Look at the date ranges of data available in this data set as I plan to merge this data set with 2 others from different sources, which are both time series data.
```
SELECT min(pos_spec_dt), max(pos_spec_dt) FROM cases_deaths;
```
| min | max |
| --- | --- |
| 2020-01-01 | 2020-09-28 |
 
It looks like the last available data point is from 09/28/2020. The data was last updated on 09/30/2020, per the website.

**National level data (The Atlantic)**

- Original dataset size: 266 rows, 19 columns

|      column_name    | data_type 
----------------------|-----------
| date                | date
| death               | integer
| death_incr          | integer
| icu_cum             | integer
| icu_curr            | integer
| hosp_incr           | integer
| hosp_curr           | integer
| hosp_cum            | integer
| negative            | integer
| neg_incr            | integer
| vent_cum            | integer
| vent_curr           | integer
| pos_neg             | integer
| positive            | integer
| pos_incr            | integer
| recovered           | integer
| states              | integer
| tot_test_results    | integer
| tot_test_results_incr | integer


- Date ranges:
```
SELECT min(date), max(date) FROM hosp;
```
|    min    |    max    |
|-----------|------------
| 2020-01-22 | 2020-10-13 |


- Look at missing values: 70 in total
```
SELECT COUNT(*) FROM hosp WHERE death IS NULL OR death IS NULL OR death_incr IS NULL OR icu_cum IS NULL OR icu_curr IS NULL OR hosp_incr IS NULL OR hosp_curr IS NULL OR hosp_cum IS NULL OR negative IS NULL OR neg_incr IS NULL OR vent_cum IS NULL OR vent_curr IS NULL OR pos_neg IS NULL OR positive IS NULL OR pos_incr IS NULL OR recovered IS NULL OR states IS NULL OR tot_test_results IS NULL OR tot_test_results_incr IS NULL;
```
- I see that the first date where there are no null values is 2020-04-01. I will start looking at all of my data sets from this point on. I went back and filtered my CDC data for 'pos_spec_dt' starting from this date:
```
SELECT * FROM cases_deaths WHERE 'pos_spec_dt' BETWEEN '2020-04-01' AND '2020-10-01';
```
count: 671435


- I want to see if there are any data points where data is not collected from every state
```
SELECT states, COUNT(*) FROM hosp GROUP BY states;
```
| states | count |
|--------|-------|
|     19 |     1 |
|      2 |    19 |
|      4 |     3 |
|     38 |     1 |
|      6 |     3 |
|     51 |     9 |
|     26 |     1 |
|      3 |    17 |
|     56 |   212 |
- It looks like there are some data points where there were no reportable cases in some states or the data was not available from some states. However, most of the data points have been collected from all 56 states included in this data collection.


- From a preliminary look at the data on the website, I noticed some discrepancies between the current and cumulative values for hospitalizations, ICU admissions, and patients on ventilator support. I also noticed some discrepancies and outliers within these columns. I want to look more closely at these discrepant/abnormal values and see if there is a pattern of the data can be kept.
```
SELECT COUNT(*) FROM hosp WHERE death_incr > death OR icu_curr > icu_cum OR hosp_curr > hosp_cum OR neg_incr > negative OR vent_curr > vent_cum OR pos_incr > positive OR tot_test_results_incr > tot_test_results OR icu_curr > hosp_curr OR icu_curr > ;
```
- There are **167** rows where something doesn't make sense! I want to figure out if a majority of these discrepancies come from rows that I can potentially drop (e.g. hosp_incr since we can calculate this ourselves with the difference between daily hospitalizations).

Keeping a count:
- icu_curr > icu_curr: 64
- hosp_curr > hosp_cum: 4
- vent_curr > vent_cum: 157
- vent_curr > icu_cum: 45

There is some overlap here. But it looks like the majority of the issues are with the subset where vent_curr > vent_cum. I will need to look at these rows in the dataframe to determine if the columns might have been swapped or if the numbers just look wrong, making the data likely unusable.

**State-level hospitalization data**

- Original data set: 12469 rows, 5 columns

|  column_name  | data_type |
|---------------|-----------|
| state         | text |
| collection_dt | date |
| inpt_beds_occ | integer |
| pct_occ       | numeric |
| tot_beds      | integer |

- Missing values: 7338
- Date ranges:
```
SELECT min(collection_dt), max(collection_dt) FROM inpatient;
```
| min        |    max     |
|------------|------------|
| 2020-01-01 | 2020-10-12 |
 
- I plan to merge this data set with the national-level hospitalization data as this will give an idea of hospitalization utilization nationwide. In preparation for this merge/join, I will transform this data set to national level data by grouping by collection dates and summing all of the numbers for the states. Since this data set contains a pct_occ column, I will take the averages of the pct_occ column as an estimator and also manually calculate the percent occupancy by dividing the sums of the # of beds occupied by the sums of the total # of beds.
```
CREATE VIEW inpatient_nat AS SELECT collection_dt, sum(inpt_beds_occ) as beds_occ, sum(inpt_beds_occ::numeric)*100/sum(tot_beds::numeric) as calc_pct_occ, avg(pct_occ) as avg_pct_occ, sum(tot_beds) as beds_tot FROM inpatient GROUP BY collection_dt;
```
Final size/info of data set: 286 rows, 5 columns

|  column_name  | data_type |
|---------------|-----------|
| collection_dt | date |
| beds_occ      | bigint |
| calc_pct_occ  | numeric |
| avg_pct_occ   | numeric |
| beds_tot      | bigint |
 
- I will rename the 'death' column to 'deaths_tot' since it has the same name as the 'death' column in the patient-level data set.
```
ALTER TABLE hosp RENAME COLUMN death TO deaths_tot;
```

## Data Cleaning

### Data set 1: Patient-level data (CDC)

Since I have >600,000 data points, I will do some preliminary data analysis on a smaller portion of my data to try to determine how I can deal with missing values, clean up the data, engineer features, etc.

I will start with the patient-level data, which is also the largest data set, to see which features are important, especially to figure out how to deal with null values.

In [281]:
query = "SELECT * FROM deaths_pos_apr ORDER BY random() LIMIT 50000;"
df = pd.read_sql(query, connection)

In [282]:
# Explore data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cdc_report_dt   50000 non-null  object
 1   pos_spec_dt     50000 non-null  object
 2   onset_dt        34768 non-null  object
 3   current_status  50000 non-null  object
 4   sex             50000 non-null  object
 5   age_group       50000 non-null  object
 6   race_ethnicity  50000 non-null  object
 7   hosp            50000 non-null  object
 8   icu             50000 non-null  object
 9   death           50000 non-null  object
 10  med_cond        50000 non-null  object
dtypes: object(11)
memory usage: 4.2+ MB


### First steps:

#### More data cleaning:
- Convert all dates to datetime objects
- Convert categorical variables to numerical variables: binary, dummy variables
- Some of the missing values are listed as 'Missing', 'Unknown', or 'NA.' I will impute 'NaN' to these values.
- Drop 'current_status' column since this column all contains the same value

#### More feature engineering:
- Extract month from primary date feature ('pos_spec_dt') as separate feature
- Some of the values for the 'onset_dt' column are null. Per the dataset documentation, this may indicate that the patient is asymptomatic. I will create a new feature ('no_sx') by imputing 0 to this new column if 'onset_dt' is null and 1 if it is not not.
- Look at durations/time periods between the different date columns

In [283]:
# Apply changes
df1 = convert(df)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   cdc_report_dt                                         50000 non-null  datetime64[ns]
 1   date                                                  50000 non-null  datetime64[ns]
 2   onset_dt                                              34768 non-null  datetime64[ns]
 3   age_group                                             49985 non-null  float64       
 4   hosp                                                  37499 non-null  float64       
 5   icu                                                   19489 non-null  float64       
 6   death                                                 50000 non-null  int64         
 7   med_cond                                              32603 non-null  float6

In [29]:
# Look at distribution of target variable
df1['death'].value_counts()

0    46698
1     3302
Name: death, dtype: int64

Note the imbalance of my target variable, 'death'. There is a ratio of ~14:1 in this data set, which is similar to the distribution of the variable in the entire data set.

In [30]:
# Look at null values
df1.isna().sum()

cdc_report_dt                                               0
date                                                        0
onset_dt                                                15366
age_group                                                  11
hosp                                                    12605
icu                                                     30665
death                                                       0
med_cond                                                17403
American Indian/Alaska Native, Non-Hispanic                 0
Asian, Non-Hispanic                                         0
Black, Non-Hispanic                                         0
Hispanic/Latino                                             0
Multiple/Other, Non-Hispanic                                0
Native Hawaiian/Other Pacific Islander, Non-Hispanic        0
White, Non-Hispanic                                         0
Female                                                      0
Male    

There are quite a number of null values. Since my target classes are imbalanced in favor of the negative class, I do not want to risk losing data so I will keep these values for now and figure out how to deal with them (e.g. drop, impute, etc.) as I continue with my data exploration and analysis and as I examine potential features.

In [31]:
df1.columns

Index(['cdc_report_dt', 'date', 'onset_dt', 'age_group', 'hosp', 'icu',
       'death', 'med_cond', 'American Indian/Alaska Native, Non-Hispanic',
       'Asian, Non-Hispanic', 'Black, Non-Hispanic', 'Hispanic/Latino',
       'Multiple/Other, Non-Hispanic',
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic',
       'White, Non-Hispanic', 'Female', 'Male', 'month', 'onset_pos_lag',
       'pos_onset_lag', 'cdc_pos_lag', 'pos_cdc_lag', 'onset_cdc_lag',
       'cdc_onset_lag', 'no_sx'],
      dtype='object')

In [52]:
# Re-order columns for ease of analysis
df1 = df1[['death', 'date', 'month', 'cdc_report_dt', 'onset_dt', 'age_group', 'hosp', 'icu',
       'med_cond', 'Female', 'Male', 'American Indian/Alaska Native, Non-Hispanic',
       'Asian, Non-Hispanic', 'Black, Non-Hispanic', 'Hispanic/Latino',
       'Multiple/Other, Non-Hispanic',
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic',
       'White, Non-Hispanic', 'onset_pos_lag',
       'pos_onset_lag', 'cdc_pos_lag', 'pos_cdc_lag', 'onset_cdc_lag',
       'cdc_onset_lag']]

In [ ]:
# Pickle df
import pickle

outfile = open('pt_df.pkl', 'wb')
pickle.dump(df1, outfile)
outfile.close()

### Data set 2: National-level data (The Atlantic)

In [325]:
# Look at national-level data to see if any data is not usable
query = "SELECT * FROM hosp ORDER BY date;"
hosp = pd.read_sql(query, connection)
hosp.head()

,date,deaths_tot,death_incr,icu_cum,icu_curr,hosp_incr,hosp_curr,hosp_cum,negative,neg_incr,vent_cum,vent_curr,pos_neg,positive,pos_incr,recovered,states,tot_test_results,tot_test_results_incr
0,2020-01-22,NaN,0,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,2,1,0
1,2020-01-23,NaN,0,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,2,2,1
2,2020-01-24,NaN,0,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,2,2,0
3,2020-01-25,NaN,0,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,2,2,0
4,2020-01-26,NaN,0,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,2,2,0


In [326]:
hosp2 = hosp.copy()

In [327]:
hosp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   266 non-null    object 
 1   deaths_tot             247 non-null    float64
 2   death_incr             266 non-null    int64  
 3   icu_cum                203 non-null    float64
 4   icu_curr               202 non-null    float64
 5   hosp_incr              266 non-null    int64  
 6   hosp_curr              211 non-null    float64
 7   hosp_cum               224 non-null    float64
 8   negative               266 non-null    int64  
 9   neg_incr               266 non-null    int64  
 10  vent_cum               196 non-null    float64
 11  vent_curr              203 non-null    float64
 12  pos_neg                266 non-null    int64  
 13  positive               266 non-null    int64  
 14  pos_incr               266 non-null    int64  
 15  recove

In [328]:
# Convert 'date' column to datetime object
hosp2['date'] = pd.to_datetime(hosp2['date'])
hosp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   266 non-null    datetime64[ns]
 1   deaths_tot             247 non-null    float64       
 2   death_incr             266 non-null    int64         
 3   icu_cum                203 non-null    float64       
 4   icu_curr               202 non-null    float64       
 5   hosp_incr              266 non-null    int64         
 6   hosp_curr              211 non-null    float64       
 7   hosp_cum               224 non-null    float64       
 8   negative               266 non-null    int64         
 9   neg_incr               266 non-null    int64         
 10  vent_cum               196 non-null    float64       
 11  vent_curr              203 non-null    float64       
 12  pos_neg                266 non-null    int64         
 13  posit

We noticed some discrepancies initially. I will look at this data set further to see what might be going on and if we can make any corrections/adjustments.

167 rows total
- icu_curr > icu_cum: 64
- hosp_curr > hosp_cum: 4
- vent_curr > vent_cum: 157
- vent_curr > icu_cum: 45

In [329]:
mask1 = hosp['icu_curr'] > hosp['icu_cum']
mask2 = hosp['hosp_curr'] > hosp['hosp_cum']
mask3 = hosp['vent_curr'] > hosp['vent_cum']
mask4 = hosp['vent_curr'] > hosp['icu_cum']

In [330]:
# Mask 1: icu_curr > icu_cum
hosp_mask1 = hosp[mask1].sort_values(by='date')
hosp_mask1

,date,deaths_tot,death_incr,icu_cum,icu_curr,hosp_incr,hosp_curr,hosp_cum,negative,neg_incr,vent_cum,vent_curr,pos_neg,positive,pos_incr,recovered,states,tot_test_results,tot_test_results_incr
64,2020-03-26,1369.0,319,91.0,1299.0,2499,7757.0,9520.0,477802,84311,NaN,258.0,0,92999,17720,97.0,56,606912,106068
65,2020-03-27,1775.0,406,124.0,1792.0,2563,10932.0,12083.0,561583,83781,NaN,324.0,0,112127,19128,2418.0,56,712098,105186
66,2020-03-28,2313.0,538,140.0,2174.0,2425,12350.0,14508.0,648958,87375,NaN,390.0,0,131819,19692,3143.0,56,810499,98401
67,2020-03-29,2834.0,521,156.0,2456.0,2775,14026.0,17283.0,717145,68187,NaN,439.0,0,151400,19581,4052.0,56,895712,85213
68,2020-03-30,3423.0,589,187.0,3087.0,2536,15772.0,19819.0,813851,96706,NaN,451.0,0,173442,22042,4560.0,56,1014722,119010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2020-05-24,93971.0,680,7801.0,8485.0,921,37483.0,184684.0,12660470,361940,639.0,4383.0,0,1648585,20072,366736.0,56,14574741,377370
124,2020-05-25,94530.0,559,7847.0,8467.0,1002,37432.0,185686.0,13065188,404718,642.0,4237.0,0,1667256,18671,379157.0,56,15007601,432860
125,2020-05-26,95175.0,645,7899.0,8585.0,17320,37561.0,203006.0,13356243,291055,650.0,4215.0,0,1684081,16825,384902.0,56,15322461,314860
126,2020-05-27,96496.0,1321,8015.0,8547.0,1713,38038.0,204719.0,13646805,290562,666.0,4222.0,0,1703528,19447,391508.0,56,15639991,317530


I also checked the values in the 'states' column for these rows to see if maybe data was not collected from all states but the data for all of the rows was collected from all states.

The 'icu_curr' column appears to be inaccurate. I will drop this column and manually calculate the increase in 'icu_cum' as a new column/feature.

In [331]:
hosp2['icu_incr'] = hosp['icu_cum'].diff()
hosp2.drop(columns='icu_curr', inplace=True)

In [332]:
hosp2.columns

Index(['date', 'deaths_tot', 'death_incr', 'icu_cum', 'hosp_incr', 'hosp_curr',
       'hosp_cum', 'negative', 'neg_incr', 'vent_cum', 'vent_curr', 'pos_neg',
       'positive', 'pos_incr', 'recovered', 'states', 'tot_test_results',
       'tot_test_results_incr', 'icu_incr'],
      dtype='object')

In [333]:
# Mask 2: hosp_curr > hosp_cum
hosp_mask2 = hosp[mask2]
hosp_mask2

,date,deaths_tot,death_incr,icu_cum,icu_curr,hosp_incr,hosp_curr,hosp_cum,negative,neg_incr,vent_cum,vent_curr,pos_neg,positive,pos_incr,recovered,states,tot_test_results,tot_test_results_incr
55,2020-03-17,123.0,23,NaN,NaN,13,325.0,55.0,65131,13945,NaN,NaN,0,10358,2588,NaN,56,89365,18628
56,2020-03-18,155.0,32,NaN,NaN,12,416.0,67.0,87199,22068,NaN,NaN,0,13447,3089,NaN,56,117646,28281
57,2020-03-19,203.0,48,NaN,NaN,18,617.0,85.0,110522,23323,NaN,NaN,0,18098,4651,NaN,56,148664,31018
58,2020-03-20,272.0,69,NaN,NaN,23,1042.0,108.0,141090,30568,NaN,NaN,0,24245,6147,NaN,56,189679,41015


The 'hosp_curr' column also looks inaccurate. I will drop it and manually calculate the increase in hospitalizations based on the 'hosp_cum' column.

*Note the dramatic increase in hospitalizations between 3/20 and 3/21. This makes sense given the pandemic's course and the course of reporting/data collection.

In [334]:
hosp2['hosp_incr'] = hosp2['hosp_cum'].diff()
hosp2.drop(columns='hosp_curr', inplace=True)

The 'vent_curr' column also looks inaccurate. I will drop this column and manually calculate increases in pts on ventilators based on 'vent_cum' column.

In [335]:
hosp2['vent_incr'] = hosp2['vent_cum'].diff()
hosp2.drop(columns='vent_curr', inplace=True)

In [336]:
# Check columns
hosp2.columns

Index(['date', 'deaths_tot', 'death_incr', 'icu_cum', 'hosp_incr', 'hosp_cum',
       'negative', 'neg_incr', 'vent_cum', 'pos_neg', 'positive', 'pos_incr',
       'recovered', 'states', 'tot_test_results', 'tot_test_results_incr',
       'icu_incr', 'vent_incr'],
      dtype='object')

I also want to look at any discrepancies/abnormalities in day-to-day differences in the calculated values: e.g. negative values.

In [337]:
mask5 = (hosp2['hosp_incr'] < 0) | (hosp2['icu_incr'] < 0) | (hosp2['vent_incr'] < 0)
hosp2[mask5]

,date,deaths_tot,death_incr,icu_cum,hosp_incr,hosp_cum,negative,neg_incr,vent_cum,pos_neg,positive,pos_incr,recovered,states,tot_test_results,tot_test_results_incr,icu_incr,vent_incr
78,2020-04-09,17999.0,2043,924.0,3923.0,59832.0,1967684,136748,39.0,0,468001,34503,24819.0,56,2475931,170255,-89.0,-177.0
85,2020-04-16,32587.0,2200,1834.0,3071.0,85951.0,2785630,133173,137.0,0,672865,30978,48851.0,56,3517236,166700,51.0,-86.0
96,2020-04-27,52653.0,1270,3720.0,3076.0,116002.0,4658012,173380,195.0,0,993356,22708,121323.0,56,5749371,201690,1149.0,-32.0
101,2020-05-02,62918.0,1562,4386.0,2031.0,135606.0,5737942,218321,375.0,0,1136954,29154,174987.0,56,7009788,260049,86.0,-1.0
111,2020-05-12,78994.0,1486,5595.0,1579.0,159536.0,8340605,286983,559.0,0,1375162,22890,238857.0,56,9901469,319634,-893.0,8.0
134,2020-06-04,104011.0,893,8787.0,-2841.0,214259.0,16987873,441895,723.0,0,1876832,20886,569623.0,56,19179791,475279,99.0,6.0
170,2020-07-10,126264.0,854,11523.0,2318.0,257571.0,35893902,742929,1118.0,0,3173216,66846,983185.0,56,39860383,838543,153.0,-20.0
206,2020-08-15,161315.0,1221,15891.0,2031.0,347045.0,61131947,693310,1663.0,0,5338901,56149,1818546.0,56,68188834,782686,127.0,-2.0
244,2020-09-22,192768.0,854,19324.0,1446.0,397801.0,86615497,672641,2130.0,0,6859731,49439,2646959.0,56,97098710,783814,-29.0,14.0
258,2020-10-06,202675.0,634,20973.0,-624.0,414461.0,97932855,722475,2388.0,0,7460634,38661,2952390.0,56,110352428,824253,161.0,18.0


*Looking at the rows above and below, these look like discrepancies. It's difficult to tell whether this is due to underreporting or error in data collection/entry. I will change these values to 0. Since these are only 10 out of 220+ rows, this will hopefully not affect the data or further analysis significantly.*

In [338]:
# Replace negative 'incr' values with 0
hosp2['hosp_incr'][hosp2['hosp_incr'] < 0] = 0
hosp2['icu_incr'][hosp2['icu_incr'] < 0] = 0
hosp2['vent_incr'][hosp2['vent_incr'] < 0] = 0

<ipython-input-338-1710bab7f18d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosp2['hosp_incr'][hosp2['hosp_incr'] < 0] = 0
<ipython-input-338-1710bab7f18d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosp2['icu_incr'][hosp2['icu_incr'] < 0] = 0
<ipython-input-338-1710bab7f18d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosp2['vent_incr'][hosp2['vent_incr'] < 0] = 0


In [339]:
# Look at 'pos_neg' column
hosp2['pos_neg'].value_counts()

0    266
Name: pos_neg, dtype: int64

Every value in this column is 0. I will drop it.

In [340]:
hosp2.drop(columns='pos_neg', inplace=True)

#### Feature engineering

In [342]:
# Calculate positive rate: total positive test results/total test results
hosp2['pos_rate'] = hosp2['positive']/hosp2['tot_test_results']

# Calculate positive increase rate: increase in positive test results/increase in total test results
hosp2['pos_incr_rate'] = hosp2['pos_incr']/hosp2['tot_test_results_incr']

# Calculate infection fatality rate: total deaths/total positive test results
hosp2['ifr'] = hosp2['deaths_tot']/hosp2['positive']

In [343]:
hosp2.columns

Index(['date', 'deaths_tot', 'death_incr', 'icu_cum', 'hosp_incr', 'hosp_cum',
       'negative', 'neg_incr', 'vent_cum', 'positive', 'pos_incr', 'recovered',
       'states', 'tot_test_results', 'tot_test_results_incr', 'icu_incr',
       'vent_incr', 'pos_rate', 'pos_incr_rate', 'ifr'],
      dtype='object')

In [344]:
# Re-order columns
hosp2 = hosp2[['date', 'deaths_tot', 'death_incr', 'ifr', 'hosp_cum', 'hosp_incr', 'icu_cum', 'icu_incr',
               'vent_cum', 'vent_incr', 'positive', 'pos_incr', 'pos_rate', 'pos_incr_rate',
               'negative', 'neg_incr', 'recovered', 'states', 'tot_test_results', 'tot_test_results_incr']]

In [345]:
# Look at ranges of values
hosp2.describe()

,deaths_tot,death_incr,ifr,hosp_cum,hosp_incr,icu_cum,icu_incr,vent_cum,vent_incr,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,states,tot_test_results,tot_test_results_incr
count,247.000000,266.000000,231.000000,224.000000,223.000000,203.000000,202.000000,196.000000,195.00000,2.660000e+02,266.000000,266.000000,241.000000,2.660000e+02,266.000000,2.030000e+02,266.000000,2.660000e+02,2.660000e+02
mean,98215.121457,780.368421,inf,227182.977679,1927.372197,11165.275862,112.237624,1182.045918,13.65641,2.644293e+06,29396.981203,0.088327,0.087476,3.002340e+07,390992.360902,1.164889e+06,47.116541,3.367486e+07,4.414289e+05
std,69814.622240,638.847872,NaN,133437.554637,1695.375222,6352.149354,137.167084,708.737963,18.16902,2.539457e+06,20528.635455,0.053404,0.060792,3.258229e+07,302937.844841,9.781330e+05,19.483461,3.645040e+07,3.434768e+05
min,0.000000,0.000000,0.010761,4.000000,0.000000,74.000000,0.000000,39.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,9.700000e+01,2.000000,1.000000e+00,0.000000e+00
25%,23012.500000,246.500000,0.028846,120642.250000,1056.000000,6842.500000,57.000000,611.250000,5.00000,1.367142e+05,17414.000000,0.070440,0.050713,6.660048e+05,83913.500000,2.485805e+05,56.000000,8.318022e+05,1.054065e+05
50%,110136.000000,739.500000,0.034854,235081.000000,1595.000000,10977.000000,86.500000,1093.500000,10.00000,1.845751e+06,27994.000000,0.079764,0.066771,1.632182e+07,404550.500000,8.943250e+05,56.000000,1.846909e+07,4.346350e+05
75%,158296.500000,1168.250000,0.053191,351742.000000,2318.000000,16742.000000,126.000000,1794.000000,16.50000,4.957378e+06,44671.250000,0.115606,0.111106,5.614258e+07,665092.500000,2.009278e+06,56.000000,6.267075e+07,7.504102e+05
max,207578.000000,2752.000000,inf,426343.000000,17320.000000,21735.000000,1149.000000,2485.000000,170.00000,7.819597e+06,76842.000000,0.192723,0.400000,1.040040e+08,977186.000000,3.124593e+06,56.000000,1.174201e+08,1.151290e+06


Given the nature of the pandemic and the data collection procession, we expect these data points to vary widely.

In [346]:
# Pickle df
import pickle

outfile = open('hosp_df.pkl', 'wb')
pickle.dump(hosp2, outfile)
outfile.close()

### Data set 3: State-level hospital utilization (CDC)

In [350]:
query = "SELECT * FROM inpatient_nat ORDER BY random();"
beds = pd.read_sql(query, connection)

In [351]:
beds.shape

(286, 5)

In [352]:
beds.head()

,collection_dt,beds_occ,calc_pct_occ,avg_pct_occ,beds_tot
0,2020-08-01,136799,9.679006,7.572321,1413358.0
1,2020-08-04,136773,9.687111,7.486071,1411907.0
2,2020-01-08,0,NaN,0.000000,NaN
3,2020-08-22,957491,67.928698,64.868491,1409553.0
4,2020-01-02,0,NaN,0.000000,NaN


In [353]:
beds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   collection_dt  286 non-null    object 
 1   beds_occ       286 non-null    int64  
 2   calc_pct_occ   93 non-null     float64
 3   avg_pct_occ    286 non-null    float64
 4   beds_tot       93 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 11.3+ KB


In [355]:
# Convert 'collection_dt' to datetime object
beds['collection_dt'] = pd.to_datetime(beds['collection_dt'])
beds.head(10)

,collection_dt,beds_occ,calc_pct_occ,avg_pct_occ,beds_tot
0,2020-08-01,136799,9.679006,7.572321,1413358.0
1,2020-08-04,136773,9.687111,7.486071,1411907.0
2,2020-01-08,0,NaN,0.000000,NaN
3,2020-08-22,957491,67.928698,64.868491,1409553.0
4,2020-01-02,0,NaN,0.000000,NaN
5,2020-02-10,0,NaN,0.000000,NaN
6,2020-02-12,0,NaN,0.000000,NaN
7,2020-06-15,82558,NaN,0.045000,NaN
8,2020-03-27,124771,NaN,0.073077,NaN
9,2020-06-30,110939,NaN,0.051887,NaN


In [356]:
beds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   collection_dt  286 non-null    datetime64[ns]
 1   beds_occ       286 non-null    int64         
 2   calc_pct_occ   93 non-null     float64       
 3   avg_pct_occ    286 non-null    float64       
 4   beds_tot       93 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 11.3 KB


Note that there are missing values in the 'calc_pct_occ' column. We can fill these in with the avg_pct_occ values.

In [357]:
beds = beds.sort_values(by='collection_dt')

In [358]:
beds['calc_pct_occ'].fillna(beds['avg_pct_occ'], inplace=True)
beds.isna().sum()

collection_dt      0
beds_occ           0
calc_pct_occ       0
avg_pct_occ        0
beds_tot         193
dtype: int64

Since calc_pct_occ is based on beds_tot, I will drop this column since it is less useful.

In [359]:
beds.drop(columns='beds_tot', inplace=True)

In [360]:
# Look at ranges of values
beds.describe()

,beds_occ,calc_pct_occ,avg_pct_occ
count,2.860000e+02,286.000000,286.000000
mean,1.846347e+05,9.151766,8.475827
std,2.862785e+05,20.736169,19.939899
min,0.000000e+00,0.000000,0.000000
25%,2.820250e+04,0.012457,0.012457
50%,1.024325e+05,0.067222,0.067222
75%,1.483950e+05,5.306603,5.186226
max,1.104846e+06,71.080442,68.131509


In [361]:
# Rename 'collection_dt' column to merge with other dataframes
beds.rename(columns={'collection_dt': 'date'}, inplace=True)
beds.columns

Index(['date', 'beds_occ', 'calc_pct_occ', 'avg_pct_occ'], dtype='object')

Note significant variation in 'beds_occ'. This is likely due to not all states being included in these reports since this data set was converted from state-level to national-level data. However, there isn't too much variation in the % occupancy, which is what we care about.

In [ ]:
# Pickle df
import pickle

outfile = open('beds_df.pkl', 'wb')
pickle.dump(beds, outfile)
outfile.close()

### Merging data sets

In [84]:
query = "SELECT * FROM deaths_pos_apr"
pt = pd.read_sql(query, connection)

In [86]:
# Apply changes
pt1 = convert(pt)

# Convert all whole numeric columns to integer types
for col, val in pt1.iteritems():
    if val.dtype == 'uint8':
        pt1[col] = pt1[col].astype(int)

pt1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671435 entries, 0 to 671434
Data columns (total 25 columns):
 #   Column                                                Non-Null Count   Dtype         
---  ------                                                --------------   -----         
 0   cdc_report_dt                                         671435 non-null  datetime64[ns]
 1   date                                                  671435 non-null  datetime64[ns]
 2   onset_dt                                              464801 non-null  datetime64[ns]
 3   age_group                                             671273 non-null  float64       
 4   hosp                                                  502065 non-null  float64       
 5   icu                                                   261080 non-null  float64       
 6   death                                                 671435 non-null  int64         
 7   med_cond                                              437337 non-

In [87]:
# Pickle df
import pickle

outfile = open('pt_df2.pkl', 'wb')
pickle.dump(pt1, outfile)
outfile.close()

In [88]:
# Import dataframes
import pickle

infile = open('hosp_df.pkl', 'rb')
hosp = pickle.load(infile)
infile.close()

hosp.head()

,date,deaths_tot,death_incr,hosp_cum,hosp_incr,icu_cum,icu_incr,vent_cum,vent_incr,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,states,tot_test_results,tot_test_results_incr
0,2020-01-22,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,NaN,0,0,NaN,2,1,0
1,2020-01-23,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0,0,NaN,2,2,1
2,2020-01-24,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,NaN,0,0,NaN,2,2,0
3,2020-01-25,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,NaN,0,0,NaN,2,2,0
4,2020-01-26,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,NaN,0,0,NaN,2,2,0


In [89]:
hosp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   266 non-null    datetime64[ns]
 1   deaths_tot             247 non-null    float64       
 2   death_incr             266 non-null    int64         
 3   hosp_cum               224 non-null    float64       
 4   hosp_incr              223 non-null    float64       
 5   icu_cum                203 non-null    float64       
 6   icu_incr               202 non-null    float64       
 7   vent_cum               196 non-null    float64       
 8   vent_incr              195 non-null    float64       
 9   positive               266 non-null    int64         
 10  pos_incr               266 non-null    int64         
 11  pos_rate               266 non-null    float64       
 12  pos_incr_rate          241 non-null    float64       
 13  negat

In [90]:
# Import dataframes
infile = open('beds_df.pkl', 'rb')
beds = pickle.load(infile)
infile.close()

beds.head()

,date,beds_occ,calc_pct_occ,avg_pct_occ
276,2020-01-01,0,0.0,0.0
67,2020-01-02,0,0.0,0.0
245,2020-01-03,0,0.0,0.0
150,2020-01-04,0,0.0,0.0
81,2020-01-05,0,0.0,0.0


In [91]:
beds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286 entries, 276 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          286 non-null    datetime64[ns]
 1   beds_occ      286 non-null    int64         
 2   calc_pct_occ  286 non-null    float64       
 3   avg_pct_occ   286 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 11.2 KB


In [92]:
# Beds has more rows/data points than hosp2 so will do left merge on these data sets
beds_hosp = beds.merge(hosp, on='date', how='left')
beds_hosp.head()

,date,beds_occ,calc_pct_occ,avg_pct_occ,deaths_tot,death_incr,hosp_cum,hosp_incr,icu_cum,icu_incr,...,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,states,tot_test_results,tot_test_results_incr
0,2020-01-01,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df2 = pt1.merge(beds_hosp, on='date', how='left')
df2.head()

,cdc_report_dt,date,onset_dt,age_group,hosp,icu,death,med_cond,"American Indian/Alaska Native, Non-Hispanic","Asian, Non-Hispanic",...,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,states,tot_test_results,tot_test_results_incr
0,2020-03-27,2020-04-01,2020-03-27,3.0,0.0,NaN,0,NaN,0,0,...,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,56.0,1240829.0,110730.0
1,2020-03-19,2020-04-01,2020-03-19,3.0,NaN,NaN,0,NaN,0,0,...,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,56.0,1240829.0,110730.0
2,2020-03-30,2020-04-01,2020-03-30,3.0,0.0,NaN,0,NaN,0,0,...,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,56.0,1240829.0,110730.0
3,2020-04-01,2020-04-01,2020-04-01,3.0,0.0,NaN,0,NaN,0,0,...,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,56.0,1240829.0,110730.0
4,2020-04-14,2020-04-01,2020-03-31,3.0,0.0,NaN,0,0.0,0,0,...,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,56.0,1240829.0,110730.0


In [362]:
df2.shape

(671435, 46)

In [363]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671435 entries, 0 to 671434
Data columns (total 46 columns):
 #   Column                                                Non-Null Count   Dtype         
---  ------                                                --------------   -----         
 0   death                                                 671435 non-null  int64         
 1   date                                                  671435 non-null  datetime64[ns]
 2   cdc_report_dt                                         671435 non-null  datetime64[ns]
 3   onset_dt                                              464801 non-null  datetime64[ns]
 4   age_group                                             671273 non-null  float64       
 5   hosp                                                  502065 non-null  float64       
 6   icu                                                   261080 non-null  float64       
 7   med_cond                                              437337 non-

In [365]:
df2.columns

Index(['death', 'date', 'cdc_report_dt', 'onset_dt', 'age_group', 'hosp',
       'icu', 'med_cond', 'American Indian/Alaska Native, Non-Hispanic',
       'Asian, Non-Hispanic', 'Black, Non-Hispanic', 'Hispanic/Latino',
       'Multiple/Other, Non-Hispanic',
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic',
       'White, Non-Hispanic', 'Female', 'Male', 'Other', 'month',
       'onset_pos_lag', 'pos_onset_lag', 'cdc_pos_lag', 'pos_cdc_lag',
       'onset_cdc_lag', 'cdc_onset_lag', 'beds_occ', 'calc_pct_occ',
       'avg_pct_occ', 'deaths_tot', 'death_incr', 'hosp_cum', 'hosp_incr',
       'icu_cum', 'icu_incr', 'vent_cum', 'vent_incr', 'positive', 'pos_incr',
       'pos_rate', 'pos_incr_rate', 'negative', 'neg_incr', 'recovered',
       'tot_test_results', 'tot_test_results_incr', 'ifr'],
      dtype='object')

## Feature Analysis

Since I have many features, I will analyze them to see which would be good to include in my models.

In [366]:
# Look at correlations to assess relationship of other features to target
df2.corr()

,death,age_group,hosp,icu,med_cond,"American Indian/Alaska Native, Non-Hispanic","Asian, Non-Hispanic","Black, Non-Hispanic",Hispanic/Latino,"Multiple/Other, Non-Hispanic",...,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,tot_test_results,tot_test_results_incr,ifr
death,1.000000,0.407548,0.519924,0.485758,0.261115,-0.005455,0.017269,0.025198,-0.061648,0.059499,...,-0.226213,-0.129699,0.303804,0.300299,-0.220826,-0.270013,-0.218049,-0.220325,-0.264327,0.041368
age_group,0.407548,1.000000,0.392678,0.241156,0.425326,-0.007235,-0.002948,0.007576,-0.151937,0.031373,...,-0.203693,-0.128368,0.280400,0.248556,-0.203294,-0.247806,-0.200404,-0.202440,-0.242964,0.068453
hosp,0.519924,0.392678,1.000000,0.554441,0.297474,0.000324,0.035981,0.084069,-0.054120,0.048970,...,-0.175912,-0.105184,0.234567,0.231190,-0.172005,-0.209927,-0.169861,-0.171570,-0.206409,0.015423
icu,0.485758,0.241156,0.554441,1.000000,0.183099,0.020300,0.027962,0.075434,-0.018988,0.035708,...,-0.112676,-0.063085,0.139997,0.142320,-0.109963,-0.126946,-0.108849,-0.109789,-0.125268,0.012749
med_cond,0.261115,0.425326,0.297474,0.183099,1.000000,0.011176,-0.025602,0.051678,-0.061241,0.015673,...,-0.079961,-0.049735,0.126872,0.133493,-0.076435,-0.111330,-0.075079,-0.076152,-0.107198,-0.007989
"American Indian/Alaska Native, Non-Hispanic",-0.005455,-0.007235,0.000324,0.020300,0.011176,1.000000,-0.012025,-0.026552,-0.040000,-0.012487,...,0.014707,0.008141,-0.015956,-0.016608,0.013879,0.015337,0.013693,0.013891,0.015021,-0.004547
"Asian, Non-Hispanic",0.017269,-0.002948,0.035981,0.027962,-0.025602,-0.012025,1.000000,-0.060759,-0.091530,-0.028573,...,-0.030913,-0.035192,0.032429,0.009971,-0.032728,-0.033984,-0.031168,-0.032473,-0.034734,0.033991
"Black, Non-Hispanic",0.025198,0.007576,0.084069,0.075434,0.051678,-0.026552,-0.060759,1.000000,-0.202106,-0.063091,...,-0.034247,-0.007362,0.048574,0.040802,-0.036040,-0.036986,-0.036837,-0.035825,-0.036153,0.014049
Hispanic/Latino,-0.061648,-0.151937,-0.054120,-0.018988,-0.061241,-0.040000,-0.091530,-0.202106,1.000000,-0.095043,...,-0.092909,-0.062317,0.004980,-0.043087,-0.095535,-0.036112,-0.094417,-0.095772,-0.042261,0.136157
"Multiple/Other, Non-Hispanic",0.059499,0.031373,0.048970,0.035708,0.015673,-0.012487,-0.028573,-0.063091,-0.095043,1.000000,...,-0.043336,-0.025839,0.062885,0.055384,-0.043326,-0.053481,-0.042651,-0.043124,-0.052269,0.014201


In [100]:
# Look at 'states' column
df2['states'].value_counts()

# This column contains the same value, I will remove it
df2.drop(columns='states', inplace=True)

In [172]:
# Make dictionary of high correlations to target
corr_d = dict(df2.corr()['death'])
high_corr = {}
for k, v in corr_d.items():
    if abs(v) >= 0.1:
        high_corr[k] = v

high_corr = dict(sorted(high_corr.items(), key=lambda kv: abs(kv[1]), reverse=True))
high_corr

{'death': 1.0,
 'hosp': 0.5199238861579197,
 'icu': 0.4857580071529307,
 'age_group': 0.40754813281550345,
 'pos_rate': 0.30380373642703634,
 'pos_incr_rate': 0.30029948237842263,
 'deaths_tot': -0.28757153782029493,
 'hosp_cum': -0.2752217815086443,
 'icu_cum': -0.2714984026919663,
 'neg_incr': -0.2700127103146776,
 'tot_test_results_incr': -0.26432730840483765,
 'med_cond': 0.261115491099446,
 'month': -0.2511259494668957,
 'vent_cum': -0.24753097320948908,
 'positive': -0.2262125473185955,
 'negative': -0.22082637542135147,
 'tot_test_results': -0.2203249987186424,
 'recovered': -0.21804919988403282,
 'death_incr': 0.20739994078567567,
 'pos_incr': -0.12969937949346416,
 'hosp_incr': 0.11646598080063608,
 'calc_pct_occ': -0.11220324732374012,
 'avg_pct_occ': -0.10652093659177901}

However, many of these features are highly colinear, some are even close to 100%. 

In [186]:
high_corr_feat = high_corr.keys()

In [187]:
# Look at how many null values are in these columns
df2[high_corr_feat].isna().sum()

death                         0
hosp                     169370
icu                      410355
age_group                   162
pos_rate                      0
pos_incr_rate                 0
deaths_tot                    0
hosp_cum                      0
icu_cum                       0
neg_incr                      0
tot_test_results_incr         0
med_cond                 234098
month                         0
vent_cum                      0
positive                      0
negative                      0
tot_test_results              0
recovered                     0
death_incr                    0
pos_incr                      0
hosp_incr                     0
calc_pct_occ                  0
avg_pct_occ                   0
dtype: int64

It looks like we will have to deal with the null values for 'hosp', 'icu', 'age_group', and 'med_cond' columns.

My next approach is to build some preliminary models with all of the features and then use backward elimination to progressively eliminate less important features.

In order to build models, I need to deal with the null values. I will use a smaller random sample of my data and run preliminary models on 2 different data sets: 1) data set with null values dropped, 2) data set with imputed values (using KNN). I will compare these iterations and see what the best approach might be.

In [191]:
# Sample portion of data set and check distribution
merged = df2.sample(50000)
merged['death'].value_counts()

0    46737
1     3263
Name: death, dtype: int64

In [192]:
# The distribution of the target is similar to my original.
# I will drop the date columns as I do not need them for analysis
merged.drop(columns={'date', 'cdc_report_dt', 'onset_dt'}, inplace=True)
merged.head(0)

,death,age_group,hosp,icu,med_cond,"American Indian/Alaska Native, Non-Hispanic","Asian, Non-Hispanic","Black, Non-Hispanic",Hispanic/Latino,"Multiple/Other, Non-Hispanic",...,vent_incr,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,tot_test_results,tot_test_results_incr


In [121]:
# Impute null values with KNNImputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
filled = imputer.fit_transform(merged)
df_filled = pd.DataFrame(filled, columns=merged.columns)
df_filled.value_counts().sum()

50000

Compare correlations:

In [123]:
merged.corr()['death']
df_filled.corr()['death']

death                                                   1.000000
age_group                                               0.406800
hosp                                                    0.526384
icu                                                     0.496966
med_cond                                                0.258346
American Indian/Alaska Native, Non-Hispanic            -0.005381
Asian, Non-Hispanic                                     0.014756
Black, Non-Hispanic                                     0.025361
Hispanic/Latino                                        -0.058036
Multiple/Other, Non-Hispanic                            0.069027
Native Hawaiian/Other Pacific Islander, Non-Hispanic   -0.008466
White, Non-Hispanic                                     0.044721
Female                                                 -0.018390
Male                                                    0.019379
Other                                                  -0.001663
month                    

death                                                   1.000000
age_group                                               0.406866
hosp                                                    0.494895
icu                                                     0.393980
med_cond                                                0.251287
American Indian/Alaska Native, Non-Hispanic            -0.005381
Asian, Non-Hispanic                                     0.014756
Black, Non-Hispanic                                     0.025361
Hispanic/Latino                                        -0.058036
Multiple/Other, Non-Hispanic                            0.069027
Native Hawaiian/Other Pacific Islander, Non-Hispanic   -0.008466
White, Non-Hispanic                                     0.044721
Female                                                 -0.018390
Male                                                    0.019379
Other                                                  -0.001663
month                    

The correlations for the higher correlated variables (e.g. 'hosp', 'icu') are somewhat different.

I will now drop the null values and re-evaluate the correlation.

In [193]:
merged_nonull = merged.dropna()

# Check distribution of target variable compared to original df
merged_nonull['death'].value_counts()
merged['death'].value_counts()

# Check correlation
merged_nonull.corr()['death']

0    12487
1      665
Name: death, dtype: int64

0    46737
1     3263
Name: death, dtype: int64

death                                                   1.000000
age_group                                               0.326197
hosp                                                    0.385706
icu                                                     0.394137
med_cond                                                0.184845
American Indian/Alaska Native, Non-Hispanic             0.016065
Asian, Non-Hispanic                                    -0.003649
Black, Non-Hispanic                                     0.043391
Hispanic/Latino                                        -0.050046
Multiple/Other, Non-Hispanic                            0.022537
Native Hawaiian/Other Pacific Islander, Non-Hispanic   -0.006703
White, Non-Hispanic                                     0.030370
Female                                                 -0.028871
Male                                                    0.030041
Other                                                        NaN
month                    

There are also notable differences in the higher correlated variables. These variables have the most missing values so this makes sense.

I will do further feature analysis via preliminary modeling to see what the best approach will be. I will compare the two engineered data sets. I will use logistic regression for a baseline and a decision tree as this type of model is more interpretable and may be useful for feature selection.

In [245]:
# Using highly correlated features
feat = ['hosp', 'icu', 'age_group', 'pos_rate', 'pos_incr_rate', 'deaths_tot', 'hosp_cum', 'icu_cum', 'neg_incr',
        'tot_test_results_incr', 'med_cond', 'month', 'vent_cum', 'positive', 'negative', 'tot_test_results',
        'recovered', 'death_incr', 'pos_incr', 'hosp_incr', 'calc_pct_occ', 'avg_pct_occ']

In [246]:
# Partition data - IMPUTED
X_i, y_i = df_filled[feat], df_filled['death']
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=5, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17, stratify=y)

In [247]:
# Partition data - DROPPED
X_d, y_d = merged_nonull[feat], merged_nonull['death']

In [296]:
# Logistic regression
lr = LogisticRegression(solver='saga', max_iter=1000)
model_scores_os(lr, X_i, y_i)
model_scores_os(lr, X_d, y_d)

/Users/sunnajo/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Accuracy score:  0.7323
Precision score:  0.16655716162943496
Recall score:  0.7836166924265843
F1 score:  0.27472229748035765
Fbeta score (beta=2):  0.45010653409090917
ROC AUC score:  0.777452076720873 

Accuracy score:  0.6822500950209046
Precision score:  0.10102156640181612
Recall score:  0.6691729323308271
F1 score:  0.17554240631163706
Fbeta score (beta=2):  0.3149327671620666
ROC AUC score:  0.6732378383910135 



/Users/sunnajo/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [319]:
# Random forest
rf = RandomForestClassifier()
model_scores_os(rf, X_i, y_i)
model_scores_os(rf, X_d, y_d)

Accuracy score:  0.924
Precision score:  0.44753946146703805
Recall score:  0.7449768160741885
F1 score:  0.5591647331786543
Fbeta score (beta=2):  0.6575716234652114
ROC AUC score:  0.9361430289333477 

Accuracy score:  0.9262637780311669
Precision score:  0.3314917127071823
Recall score:  0.45112781954887216
F1 score:  0.3821656050955413
Fbeta score (beta=2):  0.42075736325385693
ROC AUC score:  0.9115006290746883 



It looks like imputing values is much beter than dropping values in being able to classify my target variable.

In [321]:
# Look at feature importances
i_fi = model_fi(rf, X_i, y_i)
d_fi = model_fi(rf, X_d, y_d)

i_fi
d_fi

[('month', 0.00567934172607415),
 ('calc_pct_occ', 0.006464582216925742),
 ('avg_pct_occ', 0.007026402255537766),
 ('pos_incr', 0.007026447220320231),
 ('death_incr', 0.0074523170866299835),
 ('hosp_incr', 0.007909945710059449),
 ('pos_incr_rate', 0.008324488923814807),
 ('tot_test_results_incr', 0.008773908283709619),
 ('positive', 0.013144713827488282),
 ('neg_incr', 0.015339050470076799),
 ('hosp_cum', 0.01718162803502677),
 ('icu_cum', 0.018048860549222206),
 ('recovered', 0.019176295186388505),
 ('pos_rate', 0.020565982501705445),
 ('vent_cum', 0.022621025230407685),
 ('negative', 0.027916555125329364),
 ('deaths_tot', 0.031422359152068216),
 ('tot_test_results', 0.03185310176624891),
 ('icu', 0.06389035751590814),
 ('med_cond', 0.0998647596252805),
 ('hosp', 0.21287337310607868),
 ('age_group', 0.34744450448569897)]

[('month', 0.0005483641791450569),
 ('pos_incr_rate', 0.012395797526416528),
 ('pos_incr', 0.012462652406951165),
 ('avg_pct_occ', 0.012521865465240202),
 ('calc_pct_occ', 0.01313382888134298),
 ('tot_test_results_incr', 0.014079335292638835),
 ('death_incr', 0.01457620257046514),
 ('hosp_cum', 0.01564234984370307),
 ('hosp_incr', 0.015772231079346482),
 ('neg_incr', 0.01654865022732165),
 ('positive', 0.017982434209680133),
 ('icu_cum', 0.018586561723413164),
 ('negative', 0.019600688197937376),
 ('tot_test_results', 0.02020417795228195),
 ('pos_rate', 0.020738484575297624),
 ('vent_cum', 0.02210993680713377),
 ('deaths_tot', 0.02223260712695262),
 ('recovered', 0.025239537725670717),
 ('med_cond', 0.06669487646333425),
 ('icu', 0.09600558185196018),
 ('hosp', 0.21129908241456907),
 ('age_group', 0.33162475347919806)]

Since the XGBoost model can handle missing values, I will create a preliminary XGBoost model to compare the two methods. I will oversample to account for the class imbalance.

In [252]:
xgb = XGBClassifier()
model_scores_os(xgb, merged[feat], merged['death'])
model_scores_os(xgb, X_i, y_i)
model_scores_os(xgb, X_d, y_d)

Accuracy score:  0.9211
Precision score:  0.44435351882160395
Recall score:  0.8315467075038285
F1 score:  0.5792
Fbeta score (beta=2):  0.7081377151799688
ROC AUC score:  0.9582537558627372 

Accuracy score:  0.9152
Precision score:  0.42298850574712643
Recall score:  0.8531684698608965
F1 score:  0.5655737704918032
Fbeta score (beta=2):  0.708964808630876
ROC AUC score:  0.9609498873895275 

Accuracy score:  0.9251235271759787
Precision score:  0.3709677419354839
Recall score:  0.6917293233082706
F1 score:  0.48293963254593186
Fbeta score (beta=2):  0.5897435897435898
ROC AUC score:  0.9289943232781714 



It looks like XGBoost handles the missing values here well, performing similarly to the model run with the imputed data set. Since I suspect that the columns/features containing missing values are important in classifying my target variable, XGBoost looks to be a good option for modeling.

In [253]:
model_fi(xgb, merged[feat], merged['death'])
model_fi(xgb, X_i, y_i)
model_fi(xgb, X_d, y_d)

[('pos_onset_lag', 0.0),
 ('pos_cdc_lag', 0.0),
 ('cdc_onset_lag', 0.0),
 ('Female', 0.007813597),
 ('Native Hawaiian/Other Pacific Islander, Non-Hispanic', 0.0094065415),
 ('Multiple/Other, Non-Hispanic', 0.014261167),
 ('onset_pos_lag', 0.015737098),
 ('cdc_pos_lag', 0.016269965),
 ('onset_cdc_lag', 0.019756606),
 ('White, Non-Hispanic', 0.020718912),
 ('Hispanic/Latino', 0.02204749),
 ('Asian, Non-Hispanic', 0.024515174),
 ('Black, Non-Hispanic', 0.028929392),
 ('med_cond', 0.031104747),
 ('month', 0.03143656),
 ('American Indian/Alaska Native, Non-Hispanic', 0.03279717),
 ('Male', 0.034563947),
 ('icu', 0.035485815),
 ('hosp', 0.28298813),
 ('age_group', 0.3721677)]

[('month', 0.0),
 ('positive', 0.0),
 ('negative', 0.0),
 ('tot_test_results', 0.0),
 ('recovered', 0.0),
 ('icu_cum', 0.014997092),
 ('hosp_cum', 0.017266903),
 ('calc_pct_occ', 0.018997097),
 ('vent_cum', 0.021029975),
 ('med_cond', 0.02275509),
 ('death_incr', 0.023824511),
 ('deaths_tot', 0.024323754),
 ('pos_incr', 0.025007145),
 ('icu', 0.025563609),
 ('hosp_incr', 0.02628401),
 ('pos_incr_rate', 0.026328668),
 ('neg_incr', 0.03311046),
 ('tot_test_results_incr', 0.034800906),
 ('pos_rate', 0.039777517),
 ('avg_pct_occ', 0.047056228),
 ('hosp', 0.23273696),
 ('age_group', 0.3661401)]

[('month', 0.0),
 ('positive', 0.0),
 ('negative', 0.0),
 ('tot_test_results', 0.0),
 ('recovered', 0.0),
 ('icu_cum', 0.0005396995),
 ('hosp_cum', 0.015438947),
 ('death_incr', 0.017475992),
 ('vent_cum', 0.01781014),
 ('pos_incr', 0.020136394),
 ('pos_incr_rate', 0.022493633),
 ('calc_pct_occ', 0.023625657),
 ('neg_incr', 0.026530655),
 ('hosp_incr', 0.026869738),
 ('med_cond', 0.028537568),
 ('pos_rate', 0.028765619),
 ('deaths_tot', 0.02953905),
 ('avg_pct_occ', 0.030470446),
 ('tot_test_results_incr', 0.034116093),
 ('icu', 0.040514655),
 ('age_group', 0.08766119),
 ('hosp', 0.54947466)]

From my feature analysis thus far, I am wondering if my patient-level data set may be all I need. I will run preliminary models using only data from this data set and determine if this is the case.

In [219]:
query = "SELECT * FROM deaths_pos ORDER BY random() LIMIT 50000;"
pt_only = pd.read_sql(query, connection)

In [223]:
# Apply changes
pt_df = convert(pt_only)

for col, val in pt_df.iteritems():
    if val.dtype == 'uint8':
        pt_df[col] = pt_df[col].astype(int)

pt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   cdc_report_dt                                         50000 non-null  datetime64[ns]
 1   date                                                  16320 non-null  datetime64[ns]
 2   onset_dt                                              36939 non-null  datetime64[ns]
 3   age_group                                             49984 non-null  float64       
 4   hosp                                                  40394 non-null  float64       
 5   icu                                                   9504 non-null   float64       
 6   death                                                 50000 non-null  int64         
 7   med_cond                                              15058 non-null  float6

In [226]:
# Rearrange columns, drop unneeded columns
pt_df = pt_df[['death', 'age_group', 'hosp', 'icu', 'med_cond', 'American Indian/Alaska Native, Non-Hispanic',
       'Asian, Non-Hispanic', 'Black, Non-Hispanic', 'Hispanic/Latino',
       'Multiple/Other, Non-Hispanic',
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic',
       'White, Non-Hispanic', 'Female', 'Male', 'month', 'onset_pos_lag',
       'pos_onset_lag', 'cdc_pos_lag', 'pos_cdc_lag', 'onset_cdc_lag',
       'cdc_onset_lag']]

In [228]:
# Sample of patient-level data
X_pt, y_pt = pt_df.iloc[:,1:], pt_df.iloc[:,0]

In [229]:
# Impute null values with KNNImputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
X_pt_filled = imputer.fit_transform(X_pt)

# Create new df containing filled values
X_pt_filled = pd.DataFrame(X_pt_filled, columns=X_pt.columns)

In [230]:
# Logistic regression
lr_pt = LogisticRegression(solver='saga')
model_scores_os(lr_pt, X_pt_filled, y_pt)

Accuracy score:  0.8721
Precision score:  0.3000570450656018
Recall score:  0.9100346020761245
F1 score:  0.4513084513084513
Fbeta score (beta=2):  0.6469864698646985
ROC AUC score:  0.9519241574787418 



/Users/sunnajo/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [231]:
model_coef(lr_pt, X_pt_filled, y_pt)

/Users/sunnajo/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


,0
age_group,0.610523
hosp,1.829439
icu,1.036826
med_cond,0.251574
"American Indian/Alaska Native, Non-Hispanic",0.201215
"Asian, Non-Hispanic",0.025947
"Black, Non-Hispanic",0.196048
Hispanic/Latino,-0.220383
"Multiple/Other, Non-Hispanic",-0.111592
"Native Hawaiian/Other Pacific Islander, Non-Hispanic",-0.014966


In [232]:
# Random Forest
rf_pt = RandomForestClassifier()
model_scores_os(rf_pt, X_pt_filled, y_pt)

# Feature importance
for feature in zip(X_pt_filled.columns, rf_pt.feature_importances_):
    print(feature)

Accuracy score:  0.9287
Precision score:  0.42606790799561883
Recall score:  0.6730103806228374
F1 score:  0.5217974513749161
Fbeta score (beta=2):  0.6031007751937985
ROC AUC score:  0.9279514226807757 

('age_group', 0.2721571496807365)
('hosp', 0.2201446713367818)
('icu', 0.06832816414156447)
('med_cond', 0.08818257250397206)
('American Indian/Alaska Native, Non-Hispanic', 0.0020803455873300777)
('Asian, Non-Hispanic', 0.002180082988757927)
('Black, Non-Hispanic', 0.005932449743366589)
('Hispanic/Latino', 0.005424407014386835)
('Multiple/Other, Non-Hispanic', 0.0017844114967482732)
('Native Hawaiian/Other Pacific Islander, Non-Hispanic', 0.00018598757912983782)
('White, Non-Hispanic', 0.008999977041374667)
('Female', 0.00568197393883221)
('Male', 0.005630872801822405)
('month', 0.0807527382774121)
('onset_pos_lag', 0.0357845952309758)
('pos_onset_lag', 0.034993158847202874)
('cdc_pos_lag', 0.03106281470331862)
('pos_cdc_lag', 0.0329344224838323)
('onset_cdc_lag', 0.04649581458519304

In [233]:
# XGB
xgb_pt = XGBClassifier()
model_scores_os(xgb_pt, X_pt, y_pt)

# Feature importance
for feature in zip(X_pt.columns, xgb_pt.feature_importances_):
    print(feature)

Accuracy score:  0.9127
Precision score:  0.38679969301611666
Recall score:  0.8719723183391004
F1 score:  0.5358851674641149
Fbeta score (beta=2):  0.6970954356846474
ROC AUC score:  0.9580626105874568 

('age_group', 0.3721677)
('hosp', 0.28298813)
('icu', 0.035485815)
('med_cond', 0.031104747)
('American Indian/Alaska Native, Non-Hispanic', 0.03279717)
('Asian, Non-Hispanic', 0.024515174)
('Black, Non-Hispanic', 0.028929392)
('Hispanic/Latino', 0.02204749)
('Multiple/Other, Non-Hispanic', 0.014261167)
('Native Hawaiian/Other Pacific Islander, Non-Hispanic', 0.0094065415)
('White, Non-Hispanic', 0.020718912)
('Female', 0.007813597)
('Male', 0.034563947)
('month', 0.03143656)
('onset_pos_lag', 0.015737098)
('pos_onset_lag', 0.0)
('cdc_pos_lag', 0.016269965)
('pos_cdc_lag', 0.0)
('onset_cdc_lag', 0.019756606)
('cdc_onset_lag', 0.0)


My models do not perform better on patient-only data. I will move on with the entire merged data set.

### Final data cleaning

Let's take a last look at our merged dataframe and see what final data cleaning we need to do before we start analysis and modeling.

In [298]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671435 entries, 0 to 671434
Data columns (total 45 columns):
 #   Column                                                Non-Null Count   Dtype         
---  ------                                                --------------   -----         
 0   death                                                 671435 non-null  int64         
 1   date                                                  671435 non-null  datetime64[ns]
 2   cdc_report_dt                                         671435 non-null  datetime64[ns]
 3   onset_dt                                              464801 non-null  datetime64[ns]
 4   age_group                                             671273 non-null  float64       
 5   hosp                                                  502065 non-null  float64       
 6   icu                                                   261080 non-null  float64       
 7   med_cond                                              437337 non-

We still have missing null values but now I have a better idea of what to do, especially as I know that some features are likely not important and will not be included in my model. I will approach each column differently:

- I don't need to worry about the actual dates as I will not be using them as features in my analysis
- I will fill all null lag times with 0 as the only date column containing null values is 'onset_dt' and this value may have been left null if the patient did not have symptoms.
- It looks like all of the 'vent_incr' null values belong to the earliest dates so I will impute them with 0
- Since 'hosp', 'icu', 'age_group', and 'med_cond' are important features, I will leave the null values alone for now and run preliminary models to compare imputation vs. leaving the null values and using a model that can handle missing values

In [305]:
# Sort by date first
df2 = df2.sort_values(by='date').reset_index(drop=True)
df2.head(5)

,death,date,cdc_report_dt,onset_dt,age_group,hosp,icu,med_cond,"American Indian/Alaska Native, Non-Hispanic","Asian, Non-Hispanic",...,vent_incr,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,tot_test_results,tot_test_results_incr
0,0,2020-04-01,2020-03-27,2020-03-27,3.0,0.0,NaN,NaN,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
1,0,2020-04-01,2020-03-20,2020-03-20,6.0,0.0,NaN,1.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
2,0,2020-04-01,2020-03-26,2020-03-26,6.0,0.0,NaN,0.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
3,0,2020-04-01,2020-04-05,2020-03-30,6.0,1.0,1.0,0.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
4,0,2020-04-01,2020-04-03,2020-03-28,6.0,1.0,1.0,1.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0


It looks like all of these remaining missing values are at the top of the dataframe with no preceding real values. I will fill these in with 0.

In [307]:
df2[df2['vent_incr'].isna()]

,death,date,cdc_report_dt,onset_dt,age_group,hosp,icu,med_cond,"American Indian/Alaska Native, Non-Hispanic","Asian, Non-Hispanic",...,vent_incr,positive,pos_incr,pos_rate,pos_incr_rate,negative,neg_incr,recovered,tot_test_results,tot_test_results_incr
0,0,2020-04-01,2020-03-27,2020-03-27,3.0,0.0,NaN,NaN,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
1,0,2020-04-01,2020-03-20,2020-03-20,6.0,0.0,NaN,1.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
2,0,2020-04-01,2020-03-26,2020-03-26,6.0,0.0,NaN,0.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
3,0,2020-04-01,2020-04-05,2020-03-30,6.0,1.0,1.0,0.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
4,0,2020-04-01,2020-04-03,2020-03-28,6.0,1.0,1.0,1.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,0,2020-04-01,2020-03-26,2020-03-26,3.0,0.0,0.0,0.0,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
4537,0,2020-04-01,2020-04-14,2020-03-29,3.0,0.0,0.0,NaN,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
4538,0,2020-04-01,2020-04-07,2020-03-30,3.0,0.0,0.0,NaN,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0
4539,0,2020-04-01,2020-03-28,2020-03-28,3.0,NaN,NaN,NaN,0,0,...,NaN,224086.0,25791.0,0.180594,0.232918,984489.0,82981.0,7084.0,1240829.0,110730.0


In [316]:
# Fill in remaining null values with 0
df2[['onset_pos_lag', 'pos_onset_lag', 'onset_cdc_lag', 'cdc_onset_lag', 'vent_incr']].fillna(0, inplace=True)
df2.isna().sum()

/Users/sunnajo/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:4316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


death                                                        0
date                                                         0
cdc_report_dt                                                0
onset_dt                                                206634
age_group                                                  162
hosp                                                    169370
icu                                                     410355
med_cond                                                234098
American Indian/Alaska Native, Non-Hispanic                  0
Asian, Non-Hispanic                                          0
Black, Non-Hispanic                                          0
Hispanic/Latino                                              0
Multiple/Other, Non-Hispanic                                 0
Native Hawaiian/Other Pacific Islander, Non-Hispanic         0
White, Non-Hispanic                                          0
Female                                                 

In [310]:
df2.shape

(671435, 45)

In [311]:
df2.columns

Index(['death', 'date', 'cdc_report_dt', 'onset_dt', 'age_group', 'hosp',
       'icu', 'med_cond', 'American Indian/Alaska Native, Non-Hispanic',
       'Asian, Non-Hispanic', 'Black, Non-Hispanic', 'Hispanic/Latino',
       'Multiple/Other, Non-Hispanic',
       'Native Hawaiian/Other Pacific Islander, Non-Hispanic',
       'White, Non-Hispanic', 'Female', 'Male', 'Other', 'month',
       'onset_pos_lag', 'pos_onset_lag', 'cdc_pos_lag', 'pos_cdc_lag',
       'onset_cdc_lag', 'cdc_onset_lag', 'beds_occ', 'calc_pct_occ',
       'avg_pct_occ', 'deaths_tot', 'death_incr', 'hosp_cum', 'hosp_incr',
       'icu_cum', 'icu_incr', 'vent_cum', 'vent_incr', 'positive', 'pos_incr',
       'pos_rate', 'pos_incr_rate', 'negative', 'neg_incr', 'recovered',
       'tot_test_results', 'tot_test_results_incr'],
      dtype='object')

In [313]:
df2.shape

(671435, 45)

In [197]:
# Pickle final df
outfile = open('merged_df2.pkl', 'wb')
pickle.dump(df2, outfile)
outfile.close()

In [314]:
# Close postgres connection
connection.close()